In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import requests
import chardet
import xmltodict
import json
from io import BytesIO
from tqdm import tqdm
import os
import re
import markdown

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta
from dateutil import parser
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font

import base64

import feedparser

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

##### 구글빅쿼리 설정

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today().strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제가 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')

In [7]:
create_bigquery_client(bigquerykey_path).list_tables(f"{'mido-project-426906'}.{'g2b'}")

### 나라장터 입찰공사공고

In [8]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoCnstwkPPSSrch01"
params = {
    "numOfRows": "100",
    "pageNo": "1",
    "ServiceKey": "mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==",
    "inqryDiv": "1",
    "inqryBgnDt": "202404010000",
    "inqryEndDt": "202404302359",
    "bidNtceNm": "인조잔디",
    "type": "json"  # JSON 형식으로 응답을 요청합니다.
}

# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    data = response.json()
    
    # 현재 페이지의 items 가져오기
    items = data['response']['body']['items']
    
    # items가 비어있으면 반복 종료
    if not items:
        break
    
    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
df1 = pd.DataFrame(all_items)

# 데이터프레임 출력
print(df1)

KeyboardInterrupt: 

### 나라장터 입찰용역공고

In [ ]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoServcPPSSrch01"
params = {
    "numOfRows": "100",
    "pageNo": "1",
    "ServiceKey": "mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==",
    "inqryDiv": "1",
    "inqryBgnDt": "202404010000",
    "inqryEndDt": "202404302359",
    "bidNtceNm": "인조잔디",
    "type": "json"  # JSON 형식으로 응답을 요청합니다.
}

# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    data = response.json()
    
    # 현재 페이지의 items 가져오기
    items = data['response']['body']['items']
    
    # items가 비어있으면 반복 종료
    if not items:
        break

    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_items)

# 데이터프레임 출력
print(df)

In [ ]:
df[['bidNtceNo', 'bidNtceOrd', 'reNtceYn', 'rgstTyNm', 'ntceKindNm',
       'intrbidYn', 'bidNtceDt', 'refNo', 'bidNtceNm', 'ntceInsttCd',
       'ntceInsttNm', 'dminsttCd', 'dminsttNm', 'bidMethdNm',
       'cntrctCnclsMthdNm', 'ntceInsttOfclNm', 'ntceInsttOfclTelNo',
       'ntceInsttOfclEmailAdrs', 'exctvNm', 'bidQlfctRgstDt',
       'cmmnSpldmdAgrmntRcptdocMethd', 'cmmnSpldmdAgrmntClseDt',
       'cmmnSpldmdCorpRgnLmtYn', 'bidBeginDt', 'bidClseDt', 'opengDt',
       'ntceSpecDocUrl1', 'ntceSpecFileNm1', 'rbidPermsnYn',
       'arsltApplDocRcptMthdNm', 'prearngPrceDcsnMthdNm', 'totPrdprcNum',
       'drwtPrdprcNum', 'presmptPrce', 'opengPlce', 'bidNtceDtlUrl',
       'bidNtceUrl', 'bidPrtcptFeePaymntYn', 'bidPrtcptFee',
       'bidGrntymnyPaymntYn', 'crdtrNm', 'untyNtceNo', 'cmmnSpldmdMethdCd',
       'cmmnSpldmdMethdNm', 'stdNtceDocUrl', 'brffcBidprcPermsnYn',
       'dsgntCmptYn', 'rsrvtnPrceReMkngMthdNm', 'orderPlanUntyNo',
       'sucsfbidLwltRate', 'rgstDt', 'bfSpecRgstNo', 'sucsfbidMthdCd',
       'sucsfbidMthdNm', 'chgDt', 'linkInsttNm', 'dminsttOfclEmailAdrs',
       'indstrytyLmtYn', 'd2bMngRgnLmtYn', 'd2bMngPblctPlceNm', 'd2bMngBssamt',
       'd2bMngRgstEvalExmpYn', 'd2bMngAssmntUplmtRt', 'd2bMngAssmntLwstlmtRt',
       'd2bMngProgrsSttusNm', 'd2bMngNgttnPlanDate', 'd2bMngNgttnStleNm',
       'd2bMngDmndYear', 'chgNtceRsn', 'rbidOpengDt', 'VAT', 'indutyVAT',
       'rgnLmtBidLocplcJdgmBssCd', 'rgnLmtBidLocplcJdgmBssNm']]

### 나라장터 입찰물품공고

In [ ]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoThngPPSSrch01"
params = {
    "numOfRows": "100",
    "pageNo": "1",
    "ServiceKey": "mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==",
    "inqryDiv": "1",
    "inqryBgnDt": "202404010000",
    "inqryEndDt": "202404302359",
    "bidNtceNm": "인조잔디",
    "type": "json"  # JSON 형식으로 응답을 요청합니다.
}

# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    data = response.json()
    
    # 현재 페이지의 items 가져오기
    items = data['response']['body']['items']
    
    # items가 비어있으면 반복 종료
    if not items:
        break
    
    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
df3 = pd.DataFrame(all_items)

# 데이터프레임 출력
print(df3)

In [ ]:
column_mapping = {
                # 예시: 'eng_column_name': '한글 컬럼 이름'
				'bidNtceNo' : '공고번호', 'bidNtceOrd' : '공고차수', 'bidNtceNm' : '공고명', 
				'reNtceYn' : '재공고여부', 'rgstTyNm' : '연계기관공고건', 'ntceKindNm' : '공고상태명',
				'intrbidYn' : '국제입찰대상여부', 'bidNtceDt' : '입찰공고일시', 
				'refNo' : '참조공고번호', 'ntceInsttCd' : '공고기관코드',
				'ntceInsttNm' : '공고기관명', 'dminsttCd' : '수요기관코드', 
				'dminsttNm' : '조달방식명', 'bidMethdNm' : '입찰방식명',
				'cntrctCnclsMthdNm' : '계약체결방법', 'ntceInsttOfclNm' : '공고담당자명',
				'ntceInsttOfclTelNo' : '공고담당자전화번호',
				'ntceInsttOfclEmailAdrs' : '공고담당자이메일', 'exctvNm' : '공고집행관명', 
				'bidQlfctRgstDt' : '입찰참가자격등록완료시점',
				'cmmnSpldmdAgrmntRcptdocMethd' : '접수방식', 
				'cmmnSpldmdAgrmntClseDt' : '공동수급협정서등록마감시점',
				'cmmnSpldmdCorpRgnLmtYn' : '공동수급협정시구성원지역제한여부', 
				'bidBeginDt' : '입찰서제출개시일시', 'bidClseDt' : '평가비율', 
				'opengDt' : '개찰수행시작일시',
				'ntceSpecDocUrl1' : '공고규격서URL', 				
				'ntceSpecFileNm1' : '공고규격파일명', 
				'rbidPermsnYn' : '재공고없이재입찰여부',
				'arsltApplDocRcptMthdNm' : '실적신청서접수방법명', 
				'prearngPrceDcsnMthdNm' : '예정가격결정방식', 'totPrdprcNum' : '총예가건수',
				'drwtPrdprcNum' : '추첨예가건', 'presmptPrce' : '예정가격결정전부가가치세제외금액',
				'opengPlce' : '개찰장소', 'bidNtceDtlUrl' : '입찰공고상세링크',
				'bidNtceUrl' : '입찰공고확인링크', 'bidPrtcptFeePaymntYn' : '입찰참가납부수수료유무',
				'bidPrtcptFee' : '입찰참가납부수수료',
				'bidGrntymnyPaymntYn' : '입찰보증금전자납부여부', 
				'crdtrNm' : '입찰보증금항목명', 'untyNtceNo' : '입찰공고구분번호',
				'cmmnSpldmdMethdCd' : '공동수급이행구분코드',
				'cmmnSpldmdMethdNm' : '공동수급이행구분명', 
				'stdNtceDocUrl' : '표준공고문URL', 'brffcBidprcPermsnYn' : '공종별지분율목록',
				'dsgntCmptYn' : '지명경쟁여부', 'rsrvtnPrceReMkngMthdNm' : '예비가격재작성방법명',
				'orderPlanUntyNo' : '공고구분코드','sucsfbidLwltRate' : '낙찰하한율', 
				'rgstDt' : '공고등록일시', 'bfSpecRgstNo' : '사전규격등록번호', 
				'sucsfbidMthdCd' : '낙찰방법코드','sucsfbidMthdNm' : '낙찰방법', 
				'chgDt' : '공고변경일시', 'linkInsttNm' : '전자조달시스템운영기관명',
				'dminsttOfclEmailAdrs' : '수요기관담당자이메일',
				'indstrytyLmtYn' : '공고입찰업종제한여부', 'd2bMngRgnLmtYn' : '공고입찰지역제한여부',
				'd2bMngPblctPlceNm' : '입찰공고게재위치', 'd2bMngBssamt' : '방사청관리기초금액',
				'd2bMngRgstEvalExmpYn' : '방사청관리등록심사면제여부', 
				'd2bMngAssmntUplmtRt' : '방사청관리사정상한율', 
				'd2bMngAssmntLwstlmtRt' : '방사청관리사정하한율',
				'd2bMngProgrsSttusNm' : '방사청관리진행상태명', 
				'd2bMngNgttnPlanDate' : '방사청관리협상계획일자', 
				'd2bMngNgttnStleNm' : '방사청관리협상형태명','d2bMngDmndYear' : '방사청관리요구년도',
				'chgNtceRsn' : '공고종류명', 'rbidOpengDt' : '재입찰개찰일시', 'VAT' : '부가가치세',
				'indutyVAT' : '주공종부가가치세'
				
                # 필요한 경우 추가
            }

In [ ]:
len(column_mapping.keys())

### 나라장터 입찰기타공고

In [ ]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoEtcPPSSrch01"
params = {
    "numOfRows": "100",
    "pageNo": "1",
    "ServiceKey": "mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==",
    "inqryDiv": "1",
    "inqryBgnDt": "202404010000",
    "inqryEndDt": "202404302359",
    "bidNtceNm": "인조잔디",
    "type": "json"  # JSON 형식으로 응답을 요청합니다.
}

# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    data = response.json()
    
    # 현재 페이지의 items 가져오기
    items = data['response']['body']['items']
    
    # items가 비어있으면 반복 종료
    if not items:
        break
    
    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_items)

# 데이터프레임 출력
print(df)

In [ ]:
data

### 나라장터 입찰외자공고

In [ ]:
urls = ["https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoCnstwkPPSSrch01",
        "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoServcPPSSrch01",
        "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoThngPPSSrch01",
        "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoEtcPPSSrch01",
        "https://apis.data.go.kr/1230000/BidPublicInfoService04/getBidPblancListInfoFrgcptPPSSrch01"]

# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

for url in tqdm(urls):
    try:
        params = {
            "numOfRows": "100",
            "pageNo": "1",
            "ServiceKey": "mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==",
            "inqryDiv": "1",
            "inqryBgnDt": "202404010000",
            "inqryEndDt": "202404302359",
            "bidNtceNm": "인조잔디",
            "type": "json"  # JSON 형식으로 응답을 요청합니다.
        }
    
       
        # 페이지를 바꿔가며 데이터를 가져오기
        page_no = 1
        while True:
            params['pageNo'] = str(page_no)
            response = requests.get(url, params=params)
            data = response.json()
            
            # 현재 페이지의 items 가져오기
            items = data['response']['body']['items']
            
            # items가 비어있으면 반복 종료
            if not items:
                break
            
            # 현재 페이지의 items를 all_items에 추가
            all_items.extend(items)
            
            # 다음 페이지로 이동
            page_no += 1
    except:
        pass
    
# 모든 items 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_items)

In [ ]:
column_mapping = {
     'totPrdprcNum': '공고총예가건',
     'dlvrReqChgOrd': '납품요구변경차수',
     'dlvrReqRcptDate': '납품요구접수일자',
     'prdctSno': '물품순번',
     'prdctClsfcNo': '물품분류번호',
     'prdctClsfcNoNm': '품명',
     'dtilPrdctClsfcNo': '세부물품분류번호',
     'dtilPrdctClsfcNoNm': '세부품명',
     'prdctIdntNo': '물품식별번호',
     'prdctIdntNoNm': '품목',
     'prdctUprc': '단가',
     'prdctUnit': '단위',
     'prdctQty': '수량',
     'prdctAmt': '금액',
     'dlvrTmlmtDate': '납품기한일자',
     'cntrctCnclsStleNm': '계약구분',
     'exclcProdctYn': '우수제품여부',
     'optnDivCdNm': '옵션구분',
     'dminsttCd': '수요기관코드',
     'dminsttNm': '수요기관명',
     'dmndInsttDivNm': '수요기관구분',
     'dminsttRgnNm': '수요기관지역명',
     'corpNm': '업체명',
     'fnlDlvrReqYn': '최종납품요구여부',
     'incdecQty': '증감납품요구수량',
     'incdecAmt': '증감납품요구금액',
     'cntrctCorpBizno': '업체사업자등록번호',
     'dlvrReqNm': '납품요구건명',
     'cntrctNo': '계약번호',
     'cntrctChgOrd': '계약변경차수',
     'masYn': '다수공급자계약여부',
     'cnstwkMtrlDrctPurchsObjYn': '공사용자재직접구매대상여부',
     'smetprCmptProdctYn': '중소기업자간경쟁제품여부',
     'IntlCntrctDlvrReqDate': '최초납품요구접수일자',
     'dlvrReqQty': '납품요구수량',
     'dlvrReqAmt': '납품요구금액',
     'corpEntrprsDivNmNm': '업체기업구분명',
     'brnofceNm': '납품요구지청명'
}

### 나라장터 종합쇼핑몰 납품 상세 내역

In [8]:
# API 엔드포인트 및 쿼리 파라미터 설정
url = 'https://apis.data.go.kr/1230000/ShoppingMallPrdctInfoService06/getDlvrReqDtlInfoList01'
params = {
    'ServiceKey': 'mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==',
    'pageNo': '1',
    'numOfRows': '100',
    'type': 'json',
    'inqryDiv': '1',
    'inqryBgnDate': today[:-2]+'01', ## 해당월
    'inqryEndDate': today, ## 오늘날짜
    'prdctClsfcNoNm': '인조잔디'
}


# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    data = response.json()
    
    # 현재 페이지의 items 가져오기
    items = data['response']['body']['items']
    
    # items가 비어있으면 반복 종료
    if not items:
        break
    
    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
shop_detail_df = pd.DataFrame(all_items)

# 컬럼명을 한글로 변경
column_mapping = {
    'dlvrReqNo': '납품요구번호',
    'dlvrReqChgOrd': '납품요구변경차수',
    'dlvrReqRcptDate': '납품요구접수일자',
    'prdctSno': '물품순번',
    'prdctClsfcNo': '물품분류번호',
    'prdctClsfcNoNm': '품명',
    'dtilPrdctClsfcNo': '세부물품분류번호',
    'dtilPrdctClsfcNoNm': '세부품명',
    'prdctIdntNo': '물품식별번호',
    'prdctIdntNoNm': '품목',
    'prdctUprc': '단가',
    'prdctUnit': '단위',
    'prdctQty': '수량',
    'prdctAmt': '금액',
    'dlvrTmlmtDate': '납품기한일자',
    'cntrctCnclsStleNm': '계약구분',
    'exclcProdctYn': '우수제품여부',
    'optnDivCdNm': '옵션구분',
    'dminsttCd': '수요기관코드',
    'dminsttNm': '수요기관명',
    'dmndInsttDivNm': '수요기관구분',
    'dminsttRgnNm': '수요기관지역명',
    'corpNm': '업체명',
    'fnlDlvrReqYn': '최종납품요구여부',
    'incdecQty': '증감납품요구수량',
    'incdecAmt': '증감납품요구금액',
    'cntrctCorpBizno': '업체사업자등록번호',
    'dlvrReqNm': '납품요구건명',
    'cntrctNo': '계약번호',
    'cntrctChgOrd': '계약변경차수',
    'masYn': '다수공급자계약여부',
    'cnstwkMtrlDrctPurchsObjYn': '공사용자재직접구매대상여부',
    'smetprCmptProdctYn': '중소기업자간경쟁제품여부',
    'IntlCntrctDlvrReqDate': '최초납품요구접수일자',
    'dlvrReqQty': '납품요구수량',
    'dlvrReqAmt': '납품요구금액',
    'corpEntrprsDivNmNm': '업체기업구분명',
    'brnofceNm': '납품요구지청명'
}

shop_detail_df.rename(columns=column_mapping, inplace=True)

# 필터링
shop_detail_df = shop_detail_df[shop_detail_df['품명'] == '인조잔디']
shop_detail_df = shop_detail_df[shop_detail_df['최종납품요구여부'] == 'Y'].reset_index(drop=True)

# 숫자형 컬럼 변환
numeric_columns = ['단가', '수량', '금액', '납품요구수량', '납품요구금액', '증감납품요구수량', '증감납품요구금액']
shop_detail_df[numeric_columns] = shop_detail_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# 최종데이터셋
shop_detail_df = shop_detail_df[['납품요구번호', '납품요구변경차수', '물품순번', '수요기관명', '납품요구접수일자', '납품요구건명', '업체명', '물품분류번호', '품명', '세부물품분류번호', '세부품명', 
                                 '물품식별번호', '품목', '단가', '단위', '수량', '금액', '납품기한일자', '계약구분', '계약번호', '계약변경차수', '다수공급자계약여부', '공사용자재직접구매대상여부', 
                                 '중소기업자간경쟁제품여부', '우수제품여부', '옵션구분', '최종납품요구여부', '최초납품요구접수일자', '납품요구수량',  '납품요구금액', '증감납품요구수량', '증감납품요구금액', 
                                 '수요기관코드', '수요기관구분', '수요기관지역명', '업체사업자등록번호', '업체기업구분명', '납품요구지청명']]

In [9]:
# 빅쿼리 적재
save_dataframe_to_bigquery(shop_detail_df,'g2b','shop_detail_df_007',bigquerykey_path)

Data inserted into table shop_detail_df_007 successfully.


In [10]:
# 전체 종합쇼핑몰 데이터 가져오기
tables = create_bigquery_client(bigquerykey_path).list_tables(f"{'mido-project-426906'}.{'g2b'}")
table_list = [table.table_id for table in tables if 'shop_detail_df' in table.table_id and 'all' not in table.table_id]

shop_df_list = []
for tb_nm in table_list:
    data = get_dataframe_from_bigquery('g2b', tb_nm,bigquerykey_path)
    shop_df_list.extend(data.to_dict('records'))

In [11]:
all_shop_df = pd.DataFrame(shop_df_list)

In [12]:
# 빅쿼리 적재
save_dataframe_to_bigquery(all_shop_df,'g2b','shop_detail_df_all',bigquerykey_path)

Data inserted into table shop_detail_df_all successfully.


### LH 전자조달

In [ ]:
# API 엔드포인트 및 쿼리 파라미터 설정
url = 'http://openapi.ebid.lh.or.kr/ebid.com.openapi.service.OpenBidInfoList.dev'
params = {
    'ServiceKey': 'mQ9FwLgvOY3RCjUPiBiGw0ZjjfJW1G5IWVopKvRrp/+e+FCQR8ClAQ2iNB0DFLVZbCjRY+QVE7/Wy8BbCF4O7w==',
    'pageNo': '1',
    'numOfRows': '100',
    # 'type': 'json',
    # 'inqryDiv': '1',
    'tndrbidRegDtStart': '20240501',
    'tndrbidRegDtEnd': '20240531',
    # 'prdctClsfcNoNm': '인조잔디'
}


# 모든 데이터를 저장할 빈 리스트 초기화
all_items = []

# 페이지를 바꿔가며 데이터를 가져오기
page_no = 1
while True:
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)

    if response.status_code == 200:
    
        # 응답 데이터의 인코딩을 감지
        detected_encoding = chardet.detect(response.content)['encoding']
        
        # 감지된 인코딩으로 데이터를 디코딩
        xml_string = response.content.decode(detected_encoding)
        
        # XML 문자열을 파이썬 dict로 변환
        xml_dict = xmltodict.parse(xml_string)
        
        # 현재 페이지의 items 가져오기
        try:
            items = xml_dict['response']['body']['item']
        except:
            break
        
        # # items가 비어있으면 반복 종료
        # if not items:
        #     break
    
    # 현재 페이지의 items를 all_items에 추가
    all_items.extend(items)
    
    # 다음 페이지로 이동
    page_no += 1

# 모든 items 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_items)

In [ ]:
column_mapping = {
    'bidNum' : '공고번호',
    'cstrtnJobGbNm' : '업무구분',
    'bidKind' : '공고종류(분류)',
    'bidnmKor' : '입찰공고명(국문)',
    'bidnmEng' : '입찰공고명(영문)',
    'zoneHqCd' : '담당지역본부',
    'bidProgrsStatus' : '입찰진행상태현황',
    'tndrCtrctMedCd' : '입찰계약방법',
    'tndrbidRegDt' : '입찰공고등록일',
    'presmtPrc' : '추정가격',
    'addtTax' : '부가가치세',
    'designPrc' : '설계가격',
    'fdmtlAmt' : '기초금액',
    'sunjungNm' : '낙찰자선정방법',
    'gongdongNm' : '공동수급구성방식',
    'cooperdocAcptEndDtm' : '공동수급협정서접수 마감일시',
    'tndrdocAcptBgninDtm' : '입찰서접수개시일시',
    'tndrdocAcptEndDtm' : '입찰서접수마감일시',
    'openDtm' : '개찰일시',
    'zoneRstrct1' : '참가지역1',
    'zoneRstrct2' : '참가지역2',
    'zoneRstrct3' : '참가지역3',
    'zoneRstrct4' : '참가지역4',
    'vndrrstrctNm1' : '지역의무공동업체제한 참가지역1',
    'vndrrstrctNm2' : '지역의무공동업체제한 참가지역2',
    'vndrrstrctNm3' : '지역의무공동업체제한 참가지역3',
    'vndrrstrctNm4' : '지역의무공동업체제한 참가지역4'
}

df.rename(columns=column_mapping, inplace=True)

In [ ]:
df = df[['공고번호','업무구분','공고종류(분류)','입찰공고명(국문)','입찰공고명(영문)','담당지역본부','입찰진행상태현황','입찰계약방법','입찰공고등록일',
         '추정가격','부가가치세','설계가격','기초금액','낙찰자선정방법','공동수급구성방식','공동수급협정서접수 마감일시','입찰서접수개시일시',
         '입찰서접수마감일시','개찰일시','참가지역1','참가지역2','참가지역3','참가지역4','지역의무공동업체제한 참가지역1','지역의무공동업체제한 참가지역2',
         '지역의무공동업체제한 참가지역3','지역의무공동업체제한 참가지역4']]

### 인포21

In [ ]:
# 크롬 옵션 설정
chrome_options = Options()
# chrome_options.add_argument('--headless')  # 브라우저 창을 띄우지 않음
chrome_options.add_argument('--disable-gpu')

# 크롬 드라이버 서비스 설정
service = Service(ChromeDriverManager().install())

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# 웹페이지 접속
url = 'https://infose.info21c.net/info21c/bids/list/index?bidtype=con&bid_suc=bid&page=1&per-page=100'
driver.get(url)

In [ ]:
# 로그인
driver.find_element(By.CSS_SELECTOR, '#id').send_keys('midoplus') ## ID입력
driver.find_element(By.CSS_SELECTOR, '#pass').send_keys('mido6235!!') ## password입력

driver.find_element(By.CSS_SELECTOR, '#login_btn').click()

In [ ]:
# https://infose.info21c.net/info21c/bids/list?bidtype=con&bid_suc=bid ## 공사입찰
# https://infose.info21c.net/info21c/bids/list?bidtype=ser&bid_suc=bid ## 용역입찰
# https://infose.info21c.net/info21c/bids/list?bidtype=pur&bid_suc=bid ## 구매입찰

# https://infose.info21c.net/info21c/bids/list?bidtype=con&bid_suc=suc ## 공사낙찰
# https://infose.info21c.net/info21c/bids/list?bidtype=ser&bid_suc=suc ## 용역낙찰
# https://infose.info21c.net/info21c/bids/list?bidtype=pur&bid_suc=suc ## 구매낙찰

#### 공사입찰

#### 용역입찰

In [ ]:
# 초기화면
pg=1
start_date = '2024-06-28'
end_date = '2024-06-28'
url = 'https://infose.info21c.net/info21c/bids/list/index?mode=&page='+str(pg)+'&bid_suc=bid&division=1&searchtype=&userid=midoplus&searchWord=&bidtype=ser&g2bcode=&itemcode=&mainforce_code=&keyword=&location=&locationLoc=&whereis=&bid_opt=&contract=&delay=&cancel=&ulevel=&contract_sys=&convention=&sort=-writedt&bid_kind=&word_type=&subWord=&itemcodeAll=&keywordAll=&g2bcodeAll=&whereisAll=&locationAll=&bid_optAll=&conlevel=&cdate_column=writedt&cfrom_date='+start_date+'&cto_date='+end_date+'&pageSize=100'
# url = 'https://infose.info21c.net/info21c/bids/list/index?mode=&page='+str(pg)+'&bid_suc=bid&division=1&searchtype=&userid=midoplus&searchWord=&bidtype=con&g2bcode=&itemcode=C001%2CC004%2CC021%2CC036%2CC037&mainforce_code=000%2C010&keyword=%EC%95%BC%EA%B5%AC%EC%9E%A5%2C%EC%9A%B4%EB%8F%99%EC%9E%A5%2C%EC%9D%B8%EC%A1%B0%EC%9E%94%EB%94%94%2C%EC%A1%B0%EA%B2%BD%EC%8B%9D%EC%9E%AC%2C%EC%B6%95%EA%B5%AC%EC%9E%A5%2C%ED%92%8B%EC%82%B4%EC%9E%A5&location=0&locationLoc=&whereis=g2b%2Ckepco%2Cd2b%2Clh%2Cpublic%2Cprivate%2Cetc&bid_opt=&contract=&delay=&cancel=&ulevel=&contract_sys=&convention=&sort=-writedt&bid_kind=&word_type=&subWord=&itemcodeAll=Y&keywordAll=Y&g2bcodeAll=&whereisAll=Y&locationAll=Y&bid_optAll=&conlevel=&cdate_column=writedt&cfrom_date='+start_date+'&cto_date='+end_date+'&pageSize=100&conlevel='
driver.get(url)

In [ ]:
total_cnt = int(driver.find_element(By.CLASS_NAME, 'list_top').text.split('\n')[0].split(' ')[1].replace(',','')) ## 총 입찰공고건수

In [ ]:
down_page_cnt = (total_cnt // 100) + 1 ## 반복횟수

##### elements 기준

In [ ]:
bid_info_list = []

In [ ]:
for i in tqdm(range(1,down_page_cnt+1)):
    # 웹페이지 접속
    page_no = i
    # start_date = '2024-01-01'
    # end_date = '2024-06-12'
    
    url = 'https://infose.info21c.net/info21c/bids/list/index?mode=&page='+str(page_no)+'&bid_suc=bid&division=1&searchtype=&userid=midoplus&searchWord=&bidtype=ser&g2bcode=&itemcode=&mainforce_code=&keyword=&location=&locationLoc=&whereis=&bid_opt=&contract=&delay=&cancel=&ulevel=&contract_sys=&convention=&sort=-writedt&bid_kind=&word_type=&subWord=&itemcodeAll=&keywordAll=&g2bcodeAll=&whereisAll=&locationAll=&bid_optAll=&conlevel=&cdate_column=writedt&cfrom_date='+start_date+'&cto_date='+end_date+'&pageSize=100'
    driver.get(url)
    time.sleep(1)

    rows = driver.find_elements(By.CSS_SELECTOR, '#w0 > table > tbody > tr')
    
    for j in range(len(rows)):
        cells = rows[j].find_elements(By.TAG_NAME, 'td')

        bid_nm = cells[1].text
        bid_comp = cells[2].text
        bid_est_price = cells[3].text
        bid_base_price = cells[4].text
        bid_open_date = cells[5].text
        bid_close_date = cells[6].text
        bid_date = cells[7].text
        bid_cd = cells[8].text
        bid_region = cells[9].text

        bid_info_list.append([bid_nm, bid_comp, bid_est_price, bid_base_price, bid_open_date, bid_close_date, bid_date, bid_cd, bid_region])

    time.sleep(1)

In [ ]:
# DataFrame으로 변환
df = pd.DataFrame(bid_info_list)
df.columns = ['공고명/공고번호', '발주기관', '추정가격', '기초금액', '개찰일', '투찰마감', '입력일', '업종', '지역']

In [ ]:
# 전처리
df['공고명'] = df['공고명/공고번호'].str.split('[').str[:-1].apply(('[').join)
df['공고번호'] = df['공고명/공고번호'].str.split('[').str[-1].str.split(']').str[0]
df['개찰일'] = df['개찰일'].str.split('\n').str[0]
df['개찰일'] = pd.to_datetime(df['개찰일'], format='%y.%m.%d').dt.strftime('%y.%m.%d')
df['투찰마감'] = df['투찰마감'].str.split('\n').str[0]
df['투찰마감'] = pd.to_datetime(df['투찰마감'], format='%y.%m.%d').dt.strftime('%y.%m.%d')

bid_df = df[['공고명', '공고번호', '발주기관', '추정가격', '기초금액', '개찰일', '투찰마감', '입력일', '업종', '지역']]

In [ ]:
# 필요 정보 추출
filtered_bid_df = bid_df[bid_df['업종'].str.contains('건축|건설|ENG|엔지니어링')].reset_index(drop=True)
filtered_bid_df = filtered_bid_df[~filtered_bid_df['업종'].str.contains('폐기물')].reset_index(drop=True)
# filtered_bid_df = filtered_bid_df[~filtered_bid_df['공고명'].str.contains('폐기물')].reset_index(drop=True)

# 날짜 필터링
filtered_bid_df = filtered_bid_df[filtered_bid_df['투찰마감'] > datetime.today().strftime('%y.%m.%d')].reset_index(drop=True) ## 투찰마감일이 오늘보다 지났으면 제거

In [ ]:
filtered_bid_df

In [ ]:
cond1 = '인조잔디|학교|운동장|종합운동장|공원|체육공원|체육시설|체육센터|스포츠|스포츠센터|경기장|축구장|야구장|풋살장|골프장|배드민턴장|테니스장|게이트볼장|하키장|구장' # 필요키워드
cond2 = '개축|공사|용역|조성|실시|설계|계획' # 필요키워드
except_kwd = '경관|숲|꽃길|조명|산책로|펜스|운영|복지|교육|학습|보건|행사|예방|안전|치안|창출|활성화|유아|검진|관리|조명|오염|예술|문화|급식|보상|주차장|개방' # 제외키워드

In [ ]:
info_df = filtered_bid_df[(filtered_bid_df['공고명'].str.contains(cond1) |
                           filtered_bid_df['공고명'].str.contains(cond2)) &
                           ~filtered_bid_df['공고명'].str.contains(except_kwd)]
info_df

In [ ]:
info_df['구분'] = None
info_df['담당자'] = None
info_df['연락처'] = None

info_df[['입력일','지역','발주기관','구분','담당자','연락처','공고명']]

In [ ]:
info_df

In [ ]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(info_df,'info21','info_df',bigquerykey_path)

In [ ]:
get_dataframe_from_bigquery('info21','info_df',bigquerykey_path)

### 지자체 예산서 링크

In [ ]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://www.lofin365.go.kr/lf/hub/BUDLK"

api_key = "UPDXX1000317620240618111320UXQEE"

# 요청 헤더
headers = {
    "Authorization": f"Bearer {api_key}"
}

params = {
    "Key": api_key,
    "Type": "json",
    "pIndex": 1,
    "pSize": 1000,
    "fyr": 2024,
}

In [ ]:
response = requests.get(url, headers=headers, params=params, verify=False)
data = response.json()

In [ ]:
budget_df = pd.DataFrame(data['BUDLK'][1]['row'])
budget_df.columns = ['기준년도', '지역명', '자치단체코드', '자치단체명', '링크명', 'URL']

In [ ]:
save_dataframe_to_bigquery(budget_df,'budget','budget_link_df',bigquerykey_path)

### 지자체 사업별 예산

In [20]:
# API 엔드포인트 및 기본 파라미터 설정
url = "https://www.lofin365.go.kr/lf/hub/QWGJK"

api_key = "TMGNU1000317620240618111419PWNQE"

# 요청 헤더
headers = {
    "Authorization": f"Bearer {api_key}"
    }

# 데이터 수집을 위한 빈 리스트 생성
all_data = []
page = 1
page_size = 1000

while True:
    # 요청 파라미터
    params = {
        "Key": api_key,
        "Type": "json",
        "pIndex": page,
        "pSize": page_size,
        "fyr": today[:4],
        "exe_ymd": today # ex) "exe_ymd": '20240628'
    }

    # API 요청 보내기 (certifi를 사용하여 SSL 인증서 검증)
    response = requests.get(url, headers=headers, params=params, verify=False)

    # 응답 확인
    if response.status_code == 200:
        data = response.json()

        # 데이터를 리스트에 추가
        if data:
            all_data.extend(data['QWGJK'][1]['row'])
            # 다음 페이지로 이동
            page += 1
        else:
            # 더 이상 데이터가 없으면 중지
            break
    else:
        print(f"Failed to fetch data for page {page}: {response.status_code}")
        break

# # 수집된 모든 데이터 출력
# print(all_data)

In [21]:
bid_budget_df = pd.DataFrame(all_data)
bid_budget_df

,fyr,wa_laf_cd,wa_laf_hg_nm,laf_cd,laf_hg_nm,acnt_dv_cd,acnt_dv_nm,dept_cd,dbiz_cd,dbiz_nm,...,capep,sggep,etc_amt,ep_amt,cpl_amt,fld_cd,fld_nm,ane_part_cd,part_nm,padm_laf_cd
0,2024,1100000,서울,1111000,서울종로구,100,일반회계,1105020,3000000200730003,임산부·영유아 건강관리(자체),...,0,21400000,0,6219000,21400000,090,보건,091,보건의료,3000000
1,2024,1100000,서울,1111000,서울종로구,100,일반회계,1105030,3000000200730004,의료업소 홍보 및 지도,...,0,8400000,0,5116100,8400000,090,보건,091,보건의료,3000000
2,2024,1100000,서울,1111000,서울종로구,100,일반회계,1105030,3000000200730005,응급의료 교육,...,13200000,27900000,0,18660000,54300000,090,보건,091,보건의료,3000000
3,2024,1100000,서울,1111000,서울종로구,100,일반회계,1105020,3000000200730008,임상병리 검사,...,0,83737000,0,42727640,83737000,090,보건,091,보건의료,3000000
4,2024,1100000,서울,1111000,서울종로구,100,일반회계,1105020,3000000200730009,방사선 검사,...,0,40544000,0,17216440,40544000,090,보건,091,보건의료,3000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453732,2024,4900000,제주,4900000,제주본청,437,예술인복지기금,1035010,6500000202430267,예술인 창작준비금 지원,...,300000000,0,0,0,300000000,060,문화및관광,061,문화예술,6500000
453733,2024,4900000,제주,4900000,제주본청,232,주차장사업특별회계,1055010,6500000202430269,보전지출,...,223851000,0,0,0,223851000,120,교통및물류,126,대중교통ㆍ물류등기타,6500000
453734,2024,4900000,제주,4900000,제주본청,232,주차장사업특별회계,1055010,650000020243026A,내부거래지출,...,390000000,0,0,0,390000000,120,교통및물류,126,대중교통ㆍ물류등기타,6500000
453735,2024,4900000,제주,4900000,제주본청,100,일반회계,1040020,650000020243026B,소상공인 경영지원센터 운영(민관협력형 배달앱),...,0,0,0,50000000,50000000,110,산업ㆍ중소기업및에너지,114,산업진흥ㆍ고도화,6500000


In [22]:
column_mapping = {'acnt_dv_nm':'회계구분명',
                  'dept_cd':'부서코드',
                  'dbiz_cd':'세부사업코드',
                  'dbiz_nm':'세부사업명',
                  'exe_ymd':'집행일자',
                  'bdg_cash_amt':'예산현액',
                  'bdg_ntep':'국비',
                  'capep':'시도비',
                  'sggep':'시군구비',
                  'etc_amt':'기타',
                  'ep_amt':'지출액',
                  'cpl_amt':'편성액',
                  'fld_cd':'분야코드',
                  'fld_nm':'분야명',
                  'ane_part_cd':'부문코드',
                  'part_nm':'부문명',
                  'acnt_dv_cd':'회계구분코드',
                  'fyr':'회계연도',
                  'wa_laf_cd':'지역코드',
                  'wa_laf_hg_nm':'지역명',
                  'laf_cd':'자치단체코드',
                  'laf_hg_nm':'자치단체명',
                  'padm_laf_cd':'행정자치단체코드'}

In [23]:
bid_budget_df.rename(columns=column_mapping,inplace=True)
bid_budget_df = bid_budget_df[['회계연도', '지역코드', '지역명', '자치단체코드', '자치단체명', '회계구분명', '세부사업코드', '세부사업명', '집행일자', '예산현액', '지출액', '편성액', '분야명', '부문명', '행정자치단체코드']] # '국비', '시도비', '시군구비', '기타'

In [21]:
# bid_budget_df_today = bid_budget_df.copy()
# bid_budget_df_day7 = bid_budget_df.copy()

In [24]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육|체육공원|체육시설|체육센터|스포츠|스포츠센터|놀이시설|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|cctv|LED|led|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [25]:
bid_budget_df_filter = bid_budget_df[((bid_budget_df['세부사업명'].str.contains(need_kwd1) & 
                                  bid_budget_df['세부사업명'].str.contains(need_kwd2)) &
                                  ~bid_budget_df['세부사업명'].str.contains(except_kwd)) |
                                  bid_budget_df['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [26]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(bid_budget_df_filter,'budget','budget_df_0'+today,bigquerykey_path)

Data inserted into table budget_df_020240715 successfully.


### 교육청 예산

In [21]:
edu_df = pd.read_csv('C:/py_src/midoproject/data/2024년 교육청 본예산 정리.csv')
edu_df.columns = ['시도','시군구','구분','과업명','금액','면적','예산집행']

In [22]:
save_dataframe_to_bigquery(edu_df,'edu','edu_budget',bigquerykey_path)

Data inserted into table edu_budget successfully.


### 뉴스기사스크랩핑

#### 네이버뉴스

In [ ]:
# 네이버 API 인증 정보
client_id = 'AtyNtq218QgbEcjHkYAI'
client_secret = 'JDVrsZkBtZ'

# 검색 파라미터
keyword = ['인조잔디', '학교', '초등학교', '중학교', '고등학교', '대학교', '운동장', '다목적', '종합운동장', '공원', '체육공원', '체육시설', '체육센터', '스포츠센터', '관급', '연병장', '족구장', '축구장',
           '야구장', '풋살장', '배드민턴장', '테니스장', '게이트볼장', '하키장']

display = 100  # 한 번에 가져올 검색 결과 출력 건수
sort = 'date'  # 최신순 정렬
max_results = 100  # 원하는 최대 검색 결과 건수


# 결과를 저장할 리스트
all_news = []

for kwd in tqdm(keyword):
    # 반복하여 API 요청 보내기
    for start in range(1, max_results + 1, display):
        url = f"https://openapi.naver.com/v1/search/news.json?query={kwd}&display={display}&sort={sort}&start={start}"
        headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            news_data = response.json()
            all_news.extend(news_data['items'])
        else:
            print("Error Code:", response.status_code)
            break

In [ ]:
# 뉴스 기사 링크에서 제목과 본문을 가져오기 위한 함수
def get_article_content(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('meta', property='og:title')['content']
        content = soup.find('article', {'id': 'dic_area'}).get_text(strip=True)
        return title, content, url
    else:
        return None, None, url

# URL 리스트에서 제목과 본문 가져오기
def fetch_news_content(news_links):
    results = []
    for link in tqdm(news_links):
        title, content, url = get_article_content(link)
        if title and content:
            results.append({'title': title, 'content': content, 'link': url})
    return pd.DataFrame(results)

In [ ]:
# 네이버뉴스형식 기사 추출
all_news_df = pd.DataFrame(all_news)

drop_idx = []
for i in range(len(all_news_df['link'])):
    if 'n.news.naver.com' not in all_news_df['link'][i]:
            drop_idx = drop_idx + [i]

news_df = all_news_df.drop(drop_idx).reset_index(drop=True)
news_df = news_df.drop_duplicates(['link']).reset_index(drop=True)

In [ ]:
# 링크가 있는 데이터프레임의 컬럼을 리스트로 변환
news_links = news_df['link'].tolist()

# 뉴스 기사 내용 가져오기
news_links_df = fetch_news_content(news_links)

In [ ]:
naver_news_df = pd.merge(news_df,news_links_df,how='left',on='link')

In [ ]:
naver_news_df_fin = naver_news_df[['title_y', 'content','originallink', 'link', 'pubDate']]
naver_news_df_fin.columns = ['제목', '본문', '원본링크', '네이버링크', '기사날짜']

In [ ]:
naver_news_df_fin = naver_news_df_fin.drop_duplicates() # 중복제거

# 날짜 형식 변경
naver_news_df_fin['기사날짜'] = naver_news_df_fin['기사날짜'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d %H:%M:%S'))
naver_news_df_fin = naver_news_df_fin.sort_values(['기사날짜'],ascending=False).reset_index(drop=True)

In [ ]:
cond1 = '인조잔디|학교|운동장|종합운동장|공원|체육공원|체육시설|체육센터|스포츠센터|축구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장'
cond2 = '개축|공사|용역|조성|실시|설계|계획|예산|확보|예정'
cond3 = '준공식|개장식|완공|마쳤|성황리|종료' ## 제외할 키워드

In [ ]:
naver_news_df_filter1 = naver_news_df_fin[(naver_news_df_fin['제목'].str.contains(cond1) |
                                          naver_news_df_fin['제목'].str.contains(cond2)) &
                                          ~naver_news_df_fin['제목'].str.contains(cond3)].reset_index(drop=True)

In [ ]:
naver_news_df_filter2 = naver_news_df_fin[(naver_news_df_fin['본문'].str.contains(cond1) &
                                           naver_news_df_fin['본문'].str.contains(cond2)) &
                                           ~naver_news_df_fin['본문'].str.contains(cond3)].reset_index(drop=True)

In [ ]:
naver_news_df_filter = pd.concat([naver_news_df_filter1,naver_news_df_filter2],axis=0).drop_duplicates().reset_index(drop=True)

#### 구글뉴스

In [ ]:
def fetch_google_news_rss(query, num_pages=3):
    articles = []
    for page in range(num_pages):
        start = page * 10
        rss_url = f"https://news.google.com/rss/search?q={query}&hl=ko&gl=KR&ceid=KR:ko&start={start}"
        news_feed = feedparser.parse(rss_url)
        for entry in news_feed.entries:
            article = {
                'title': entry.title,
                'link': entry.link,
                'published': entry.published,
                'summary': entry.summary
            }
            articles.append(article)
    return articles

In [ ]:
query = "인조잔디"
articles = fetch_google_news_rss(query,5)
if articles:
    for article in articles:
        print(f"Title: {article['title']}")
        print(f"Link: {article['link']}")
        print(f"Published: {article['published']}")
        print(f"Summary: {article['summary']}")
        print("-------")
else:
    print("No articles found")

In [ ]:
# 데이터셋
google_news_df = pd.DataFrame(articles)
google_news_df = google_news_df.drop_duplicates() # 중복제거

# 날짜형식변경
google_news_df['published'] = google_news_df['published'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d %H:%M:%S'))
google_news_df = google_news_df.sort_values(['published'],ascending=False).reset_index(drop=True)

# 타이틀 불필요 제거
google_news_df['title'] = google_news_df['title'].str.split('-').str[:-1].apply((' ').join)

google_news_df.columns = ['제목','원본링크','기사날짜','요약']

#### 뉴스 데이터 통합

In [ ]:
# 네이버, 구글 뉴스 통합
total_news_df = pd.concat([naver_news_df_filter[['기사날짜','제목','원본링크']], google_news_df[['기사날짜','제목','원본링크']]],axis=0)
total_news_df['기사날짜'] = total_news_df['기사날짜'].apply(parser.parse) # 날짜 형식으로 변경

total_news_df = total_news_df.sort_values('기사날짜').reset_index(drop=True)

In [ ]:
total_news_df_filter = total_news_df[total_news_df['기사날짜']>='2024-06-01']
total_news_df_filter['기사날짜'] = total_news_df_filter['기사날짜'].astype('str').str.split(' ').str[0] # 연도, 월, 일 추출

In [ ]:
news_keyword = '운동장|축구장|야구장|게이트볼|배드민턴|테니스|학교|운동시설|스포츠타운|골프|조성|확충|유지|보수|예산|계획|추진|지원|용역'

In [ ]:
total_news_df_filter[(total_news_df_filter['제목'].str.contains(cond1) |
                     total_news_df_filter['제목'].str.contains(cond2)) &
                     ~total_news_df_filter['제목'].str.contains(cond3)].reset_index(drop=True)

##### 기사에서 지역 추출

In [ ]:
dist_code = pd.read_csv('C:/py_src/midoproject/data/법정동코드 전체자료.csv',encoding='cp949')

In [ ]:
dist_code['시도'] = dist_code['법정동명'].str.split(' ').str[0]
dist_code['시군구'] = dist_code['법정동명'].str.split(' ').str[1]
dist_code['시군구'] = np.where(dist_code['시군구'].isnull(),'',dist_code['시군구'])

In [ ]:
dist_code['지역'] = np.where(dist_code['시도']=='충청북도','충북',dist_code['시도'].str[:2])
dist_code['지역'] = np.where(dist_code['시도']=='충청남도','충남',dist_code['시도'].str[:2])
dist_code['지역'] = np.where(dist_code['시도']=='전라북도','전북',dist_code['시도'].str[:2])
dist_code['지역'] = np.where(dist_code['시도']=='전라남도','전남',dist_code['시도'].str[:2])

dist_code['지역2'] = np.where(dist_code['시군구'].apply(len)<=2, dist_code['시군구'], dist_code['시군구'].str[:2])

In [ ]:
# 지역 패턴 추출
district_pat = ('|').join(dist_code['지역'].unique()) ## 시도
district_pat2 = ('|').join(dist_code['지역2'].unique()[1:]) ## 시군구

In [ ]:
# 시도를 추출하는 함수
def extract_region1(article):
    match = re.search(district_pat, article)
    return match.group(0) if match else None

# 시군구를 추출하는 함수
def extract_region2(article):
    match = re.search(district_pat2, article)
    return match.group(0) if match else None

In [ ]:
# 각 기사에서 시도, 시군구 추출
regions = [extract_region1(article) for article in total_news_df_filter['제목']]
total_news_df_filter['지역'] = regions

regions2 = [extract_region2(article) for article in total_news_df_filter['제목']]
total_news_df_filter['지역2'] = regions2

In [ ]:
# 지역명 추출
total_news_df_filter['지역명'] = np.where(total_news_df_filter['지역'].isnull(),total_news_df_filter['지역2'],total_news_df_filter['지역'])

In [ ]:
total_news_df_filter

In [ ]:
total_news_df_filter_fin = total_news_df_filter[['기사날짜','지역명','제목','원본링크']]
total_news_df_filter_fin.columns = ['date','region','title','link']

In [ ]:

save_dataframe_to_bigquery(total_news_df_filter_fin,'mido_test','news_daily',bigquerykey_path)

In [ ]:
# 엑셀 파일 생성
wb = Workbook()
ws = wb.active
# ws.title = "Linked Articles"

# 데이터프레임의 헤더를 엑셀 워크시트에 추가
for col_num, column_title in enumerate(["날짜", '지역', "제목", "링크"], 1):
    cell = ws.cell(row=1, column=col_num)
    cell.value = column_title

# 데이터프레임의 데이터를 엑셀 워크시트에 추가하며 '제목'에 하이퍼링크 설정
for row_num, row_data in enumerate(total_news_df_filter_fin.itertuples(), 2):
    ws.cell(row=row_num, column=1, value=row_data.date)  # 날짜
    ws.cell(row=row_num, column=2, value=row_data.region)  # 지역
    title_cell = ws.cell(row=row_num, column=3, value=row_data.title)  # 제목
    title_cell.hyperlink = row_data.link
    title_cell.font = Font(color="0000FF", underline="single")
    ws.cell(row=row_num, column=4, value=row_data.link)  # 링크

# 링크 열 숨기기
ws.column_dimensions['D'].hidden = True


In [ ]:
wb.save("C:/py_src/midoproject/data/news_data_daily.xlsx")